# Demo thuật toán SVM cho bài toán phân loại đơn giản

Notebook này minh họa cách dùng **Support Vector Machine (SVM)** để giải một bài toán phân loại nhị phân 2D.

## 1. SVM là gì?

SVM (Support Vector Machine) là một thuật toán **phân loại có giám sát**. Ý tưởng chính:

- Tìm một siêu phẳng (đường thẳng trong 2D) **phân tách hai lớp** dữ liệu.
- Siêu phẳng này phải có **khoảng cách biên (margin)** tới các điểm gần nhất của mỗi lớp là **lớn nhất**.
- Những điểm nằm sát biên quyết định siêu phẳng được gọi là **support vectors**.

SVM có thể dùng **kernel** để phân loại dữ liệu không tuyến tính, ví dụ:
- `linear`
- `rbf` (Radial Basis Function)
- `poly` (đa thức)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

%matplotlib inline
plt.rcParams['figure.figsize'] = (6, 6)
plt.rcParams['font.size'] = 12

## 2. Tạo dữ liệu ví dụ (make_moons)

Ta dùng hàm `make_moons` để tạo dữ liệu 2D gồm 2 lớp có dạng hình trăng lưỡi liềm, **không tuyến tính** nên phù hợp để demo SVM với kernel RBF.

In [ ]:
# Tạo dữ liệu 2 lớp
X, y = make_moons(n_samples=300, noise=0.2, random_state=42)
print('Kích thước X:', X.shape)

# Vẽ dữ liệu
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.title('Dữ liệu ban đầu (make_moons)')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

## 3. Chia dữ liệu train/test

Ta chia dữ liệu thành 2 phần:
- 80% để train (huấn luyện)
- 20% để test (đánh giá)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print('Train size:', X_train.shape[0])
print('Test size:', X_test.shape[0])

## 4. Huấn luyện mô hình SVM (kernel RBF)

Ta dùng `SVC` trong `sklearn.svm` với:
- `kernel='rbf'`
- `C=1.0` (tham số phạt lỗi)
- `gamma='scale'` (mặc định, điều khiển độ cong của biên quyết định)

In [ ]:
# Khởi tạo và huấn luyện mô hình SVM
svm_clf = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_clf.fit(X_train, y_train)

# Dự đoán trên tập test
y_pred = svm_clf.predict(X_test)

print('Độ chính xác (accuracy) trên tập test:', accuracy_score(y_test, y_pred))
print('\nBáo cáo phân loại:')
print(classification_report(y_test, y_pred))

## 5. Trực quan hóa biên quyết định (decision boundary)

Ta vẽ biên phân tách của SVM trên toàn bộ không gian 2D để thấy khả năng phân loại không tuyến tính.

In [ ]:
def plot_decision_boundary(model, X, y, title='SVM Decision Boundary'):
    # Tạo lưới điểm để vẽ contour
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 300),
        np.linspace(y_min, y_max, 300)
    )

    # Dự đoán cho từng điểm trên lưới
    grid_points = np.c_[xx.ravel(), yy.ravel()]
    Z = model.predict(grid_points)
    Z = Z.reshape(xx.shape)

    # Vẽ vùng quyết định và điểm dữ liệu
    plt.contourf(xx, yy, Z, alpha=0.3)
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolor='k')
    plt.title(title)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.show()

plot_decision_boundary(svm_clf, X, y, title='Biên quyết định của SVM (kernel RBF)')

## 6. So sánh nhanh: kernel tuyến tính (`linear`) vs RBF

Ta thử train thêm một mô hình SVM với kernel `linear` để thấy rằng dữ liệu dạng `make_moons` rất khó phân tách tuyến tính.

In [ ]:
# SVM với kernel tuyến tính
svm_linear = SVC(kernel='linear', C=1.0)
svm_linear.fit(X_train, y_train)

y_pred_linear = svm_linear.predict(X_test)
print('Accuracy (kernel linear) trên tập test:', accuracy_score(y_test, y_pred_linear))

plot_decision_boundary(svm_linear, X, y, title='Biên quyết định của SVM (kernel linear)')

## 7. Tóm tắt

Trong notebook này, bạn đã:
- Tạo dữ liệu phân loại nhị phân 2D bằng `make_moons`.
- Chia tập train/test.
- Huấn luyện SVM với kernel RBF và đánh giá độ chính xác.
- Vẽ biên quyết định của SVM để thấy trực quan khả năng phân tách không tuyến tính.
- So sánh nhanh với SVM kernel tuyến tính.

Bạn có thể thử thay đổi các tham số `C`, `gamma`, `kernel` để quan sát mô hình thay đổi như thế nào.